In [1]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'dataset/train.csv', 'val':'dataset/val.csv'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

In [2]:
dataset = dataset.remove_columns('Unnamed: 0')
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'label'],
        num_rows: 6448
    })
    val: Dataset({
        features: ['Text', 'label'],
        num_rows: 1613
    })
})

In [3]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased" # or path of the model for in local storage
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [4]:
def tokenize(batch):
    return tokenizer(batch["Text"], padding=True, truncation=True)

In [5]:
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/6448 [00:00<?, ? examples/s]

Map:   0%|          | 0/1613 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModel

model_ckpt = "distilbert-base-uncased"
device = "cuda"
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [7]:
dataset_encoded.set_format("torch", 
                            columns=["input_ids", "attention_mask", "label"])

In [8]:
from transformers import AutoModelForSequenceClassification

num_labels = 5
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [10]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(dataset_encoded["train"]) // batch_size
model_name = f"checkpoint/{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=50,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False, 
                                  log_level="error")

In [11]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=dataset_encoded["train"],
                  eval_dataset=dataset_encoded["val"],
                  tokenizer=tokenizer)

[codecarbon INFO @ 00:34:00] [setup] RAM Tracking...
[codecarbon INFO @ 00:34:00] [setup] GPU Tracking...
[codecarbon INFO @ 00:34:00] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 00:34:00] [setup] CPU Tracking...
[codecarbon WARNING @ 00:34:00] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 00:34:01] We saw that you have a 12th Gen Intel(R) Core(TM) i5-12400 but we don't know it. Please contact us.
[codecarbon INFO @ 00:34:01] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i5-12400
[codecarbon INFO @ 00:34:01] >>> Tracker's metadata:
[codecarbon INFO @ 00:34:01]   Platform system: Linux-5.15.0-100-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 00:34:01]   Python version: 3.10.10
[codecarbon INFO @ 00:34:01]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 00:34:01]   Available RAM : 125.571 GB
[codecarbon INFO @ 00:34:01]   CPU count: 12
[codecarbon INFO @ 00:34:01]   CPU model: 12th Gen Intel(R) Core(TM) i5-12400
[codecar

In [12]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: deede-rizki. Use `wandb login --relogin` to force relogin


  0%|          | 0/5050 [00:00<?, ?it/s]

[codecarbon INFO @ 00:34:31] Energy consumed for RAM : 0.000196 kWh. RAM Power : 47.089237689971924 W
[codecarbon INFO @ 00:34:31] Energy consumed for all GPUs : 0.001752 kWh. Total GPU Power : 420.53200000000004 W
[codecarbon INFO @ 00:34:31] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:34:31] 0.002126 kWh of electricity used since the beginning.


{'loss': 1.4406, 'grad_norm': 2.3573577404022217, 'learning_rate': 1.9603960396039604e-05, 'epoch': 0.99}


  0%|          | 0/26 [00:00<?, ?it/s]

...

{'loss': 0.104, 'grad_norm': 0.30626097321510315, 'learning_rate': 1.9801980198019803e-07, 'epoch': 49.5}


  0%|          | 0/26 [00:00<?, ?it/s]

[codecarbon INFO @ 00:49:12] Energy consumed for RAM : 0.011716 kWh. RAM Power : 47.089237689971924 W
[codecarbon INFO @ 00:49:12] Energy consumed for all GPUs : 0.105465 kWh. Total GPU Power : 446.619 W
[codecarbon INFO @ 00:49:12] Energy consumed for all CPUs : 0.010579 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 00:49:12] 0.127760 kWh of electricity used since the beginning.


{'eval_loss': 3.877136468887329, 'eval_accuracy': 0.4370737755734656, 'eval_f1': 0.43468557129075325, 'eval_runtime': 1.3412, 'eval_samples_per_second': 1202.677, 'eval_steps_per_second': 19.386, 'epoch': 50.0}
{'train_runtime': 904.4104, 'train_samples_per_second': 356.475, 'train_steps_per_second': 5.584, 'train_loss': 0.29483315798315674, 'epoch': 50.0}


TrainOutput(global_step=5050, training_loss=0.29483315798315674, metrics={'train_runtime': 904.4104, 'train_samples_per_second': 356.475, 'train_steps_per_second': 5.584, 'train_loss': 0.29483315798315674, 'epoch': 50.0})

In [13]:
trainer.save_model("model/dune_rate_models")

In [14]:
from transformers import pipeline

classifier = pipeline('text-classification', model='model/dune_rate_models')

result = classifier("After watching This second Time In IMAX safe to say that This is the peak sci fi sequel Ever ")
print(result)

[{'label': 'LABEL_4', 'score': 0.9998780488967896}]
